In [4]:
from datetime import datetime

import pandas as pd
import numpy as np

import codecs, json 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import scipy
import time

import functions.scipy_interpolation_functions as scipy_int

In [16]:
def read_data():    
    with open('data/data_rich_ac.bsad') as json_file:
        json_data = json.load(json_file)

    frames = []
    for j in range(len(json_data['tables'])):
        df = pd.DataFrame(np.array(json_data['tables'][j]['table'])[:,:], 
                               columns = json_data['tables'][j]['header']['variables'][:])
        df['MODE'] = json_data['tables'][j]['header']['mode']
        df['STATE'] = json_data['tables'][j]['header']['flightphase']
        if df['STATE'][0] == 'cruise':
            frames.append(df)

    df = pd.concat(frames,ignore_index=True)
    df = df[(df.MODE == 'general')]
    df = df[['DISA','ALTITUDE','MASS','MACH','FUELFLOW']]

    scaler = MinMaxScaler()
    df = scaler.fit_transform(df)

    df = pd.DataFrame(df, columns = ['DISA','ALTITUDE','MASS','MACH','FUELFLOW'])

    X = df.drop(['FUELFLOW'], axis=1)
    y = df.FUELFLOW

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001)
    return X_train, X_test, y_train, y_test

In [21]:
X_train, X_test, y_train, y_test = read_data()

In [25]:
df = pd.concat([X_train, X_test])
df.count()

DISA        73286
ALTITUDE    73286
MASS        73286
MACH        73286
dtype: int64

In [69]:
group_min = df.groupby(['DISA', 'ALTITUDE', 'MASS']).min()
group_max = df.groupby(['DISA', 'ALTITUDE', 'MASS']).max()

df_group = pd.DataFrame(index=range(len(group_min)),columns=range(5))

for i in range(len(out)):
    df_group.iloc[i] = [group_min.index[i][0],
                        group_min.index[i][1],
                        group_min.index[i][2],
                        group_min.iloc[i][0],
                        group_max.iloc[i][0]]

df_group.columns = ['DISA', 'ALTITUDE', 'MASS', 'MACH_MIN', 'MACH_MAX']

In [85]:
new_point = df_group[df_group.index == 206]
df_group_int = df_group[df_group.index != 206]

new_point_X = new_point.drop(['MACH_MIN', 'MACH_MAX'], axis=1)
new_point_y_min = new_point['MACH_MIN']
new_point_y_max = new_point['MACH_MAX'] 

df_group_int_X = df_group_int.drop(['MACH_MIN', 'MACH_MAX'], axis=1)
df_group_int_y_min = df_group_int['MACH_MIN']
df_group_int_y_max = df_group_int['MACH_MAX']

interpolated_val_min = scipy.interpolate.griddata(df_group_int_X, df_group_int_y_min, new_point_X, method='nearest')
interpolated_val_max = scipy.interpolate.griddata(df_group_int_X, df_group_int_y_max, new_point_X, method='nearest')

print(interpolated_val_min.iloc[0], ' ', new_point_y_min.iloc[0])
print(interpolated_val_max.iloc[0], ' ', new_point_y_max.iloc[0])


0.3225806451612904   0.3225806451612904
0.8706709677419355   0.8928048387096774
